# Deploy and run a remote Ollama + Llama2 endpoint
This is a google colab notebook which uses ngrok and ollama to run a public HTTP endpoint for serving Ollama models


In [ ]:
!pip install aiohttp pyngrok

In [ ]:
!ngrok config add-authtoken !!!YOUR NGROK AUTH TOKEN!!!!

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ollama', 'pull', 'llama2']),
    run_process(['ngrok', 'http', '--log', 'stderr', '11434']),
)

In [ ]:
# Prompt it
!curl https://XXXX_NGROK_IP.ngrok-free.app/api/generate -d '{ "model": "llama2","prompt": "Why is the sky blue?" }'